Ranking system for Hyper Duplex Style Alloys

In [1]:
# Imports
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Si Standard
Cr_values  = [27.0, 27.5, 28.0, 28.5, 29.0]
N_values   = [0.00, 0.05, 0.10, 0.15]
C_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.30, 0.50, 0.70]
Si_values = [9.5, 10.0, 10.5]

X_values   = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]

In [3]:
# Input data 
X_elements = ["Ni", "Mn", "Mo"]

service_temperature_C = 20

""" https://en.wikipedia.org/wiki/Prices_of_chemical_elements """
cost_usd_per_kg = {
    "Fe": 0.42,
    "C":  0.13,
    "Cr": 9.4,
    "Mo": 40.1,
    "Ni": 13.9,
    "Mn": 1.8,
    "Si": 1.7,
    "N":  0.15,
}

density_proxy = 7.8

In [4]:
# Ranking calculation proxies

def pren(Cr_wt, Mo_wt=0.0, N_wt=0.0, W_wt=0.0):
    """PREN ≈ %Cr + 3.3(%Mo + 0.5%W) + 16%N  (Si does not enter PREN)"""
    pren = Cr_wt + 3.3*(Mo_wt + 0.5*W_wt) + 16.0*N_wt
    return pren

# CHANGE FUNCTION
def strength_proxy(alloy):
    """Solid solution strengthening proxy (MPa)"""
    k = {"Cr": 35.0, "Mo": 120.0, "Ni": 20.0, "Mn": 25.0, "Si": 20.0, "C": 250.0, "N": 180.0}
    s = 0.0
    for el, wt in alloy.items():
        if el in k and wt > 0:
            s += k[el] * math.sqrt(wt)
    return s

def cost_proxy(alloy):
    """Cost proxy ($/kg)"""
    cost = 0.0
    for el, wt in alloy.items():
        el_cost = cost_usd_per_kg.get(el, 0.0)
        cost += (wt / 100.0) * el_cost
    return cost

In [5]:
# Generate data
rows = []
for X in X_elements:
    for C in C_values:
        for Cr in Cr_values:
            for N in N_values:
                for Si in Si_values:
                    for x_wt in X_values:
                        comp = {"C": C, "Cr": Cr, "N": N, "Si": Si, X: x_wt}
                        PREN = pren(Cr_wt=Cr, Mo_wt=comp.get("Mo", 0.0), N_wt=N)
                        dss  = strength_proxy(comp)

                        Fe_balance = 100.0 - (C + Cr + N + Si + x_wt)

                        rows.append({
                            "X_element": X,
                            "C_wt%": C,
                            "Cr_wt%": Cr,
                            "N_wt%": N,
                            "Si_wt%": Si,
                            f"{X}_wt%": x_wt,
                            "Fe_wt% (balance)": Fe_balance,
                            "PREN": PREN,
                            "Strength_proxy_MPa": dss,
                            "Cost_proxy_$_per_kg": cost_proxy(comp),
                            "Service_T_C (info)": service_temperature_C,
                        })

df = pd.DataFrame(rows)

csv_path = "HD_Si_Screening.csv"
df.to_csv(csv_path, index=False)
print(f"CSV saved: {csv_path}")
df.head(-10)

CSV saved: HD_Si_Screening.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Si_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Mo_wt%
0,Ni,0.01,27.0,0.00,9.5,0.0,63.49,27.00,268.509475,2.699513,20,NaN,NaN
1,Ni,0.01,27.0,0.00,9.5,0.5,62.99,27.00,282.651610,2.769013,20,NaN,NaN
2,Ni,0.01,27.0,0.00,9.5,1.0,62.49,27.00,288.509475,2.838513,20,NaN,NaN
3,Ni,0.01,27.0,0.00,9.5,1.5,61.99,27.00,293.004372,2.908013,20,NaN,NaN
4,Ni,0.01,27.0,0.00,9.5,2.0,61.49,27.00,296.793746,2.977513,20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49125,Mo,0.70,29.0,0.15,10.5,NaN,56.65,41.30,740.012979,4.108635,20,NaN,3.0
49126,Mo,0.70,29.0,0.15,10.5,NaN,56.15,42.95,756.666325,4.309135,20,NaN,3.5
49127,Mo,0.70,29.0,0.15,10.5,NaN,55.65,44.60,772.166882,4.509635,20,NaN,4.0
49128,Mo,0.70,29.0,0.15,10.5,NaN,55.15,46.25,786.725323,4.710135,20,NaN,4.5


In [6]:
df["Score"] = df["PREN"] + df["Strength_proxy_MPa"] / 100.0
df_sorted = df.sort_values(by="Score", ascending=False)
df_sorted.head(20)

,X_element,C_wt%,Cr_wt%,N_wt%,Si_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Mo_wt%,Score
49139,Mo,0.7,29.0,0.15,10.5,NaN,49.65,64.4,911.640201,6.915635,20,NaN,10.0,73.516402
49118,Mo,0.7,29.0,0.15,10.0,NaN,50.15,64.4,910.078348,6.907135,20,NaN,10.0,73.500783
49097,Mo,0.7,29.0,0.15,9.5,NaN,50.65,64.4,908.476934,6.898635,20,NaN,10.0,73.484769
47879,Mo,0.5,29.0,0.15,10.5,NaN,49.85,64.4,879.251890,6.915375,20,NaN,10.0,73.192519
47858,Mo,0.5,29.0,0.15,10.0,NaN,50.35,64.4,877.690036,6.906875,20,NaN,10.0,73.176900
47837,Mo,0.5,29.0,0.15,9.5,NaN,50.85,64.4,876.088623,6.898375,20,NaN,10.0,73.160886
48887,Mo,0.7,28.5,0.15,10.5,NaN,50.15,63.9,910.008302,6.868635,20,NaN,10.0,73.000083
48866,Mo,0.7,28.5,0.15,10.0,NaN,50.65,63.9,908.446449,6.860135,20,NaN,10.0,72.984464
48845,Mo,0.7,28.5,0.15,9.5,NaN,51.15,63.9,906.845036,6.851635,20,NaN,10.0,72.968450
46619,Mo,0.3,29.0,0.15,10.5,NaN,50.05,64.4,839.405834,6.915115,20,NaN,10.0,72.794058
